In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"
os.environ['TRANSFORMERS_CACHE'] = '/mnt/iMVR/junde/.cache/huggingface/hub'

from langchain.llms import HuggingFacePipeline
import networkx as nx
from transformers import pipeline
import re
from creat import creat_world
from prompt import *

In [4]:
pipeline = pipeline(model="meta-llama/Llama-2-7b-chat-hf", device_map="auto")
model = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.28s/it]


In [5]:
locations, agents = creat_world(model)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


You are creating a new agent: Toblen Stonehill
Name: Toblen Stonehill (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Toblen owns a trading post.


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


You are creating a new agent: Daran Edermath
Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


You are creating a new agent: Linene Graywind
Name: Linene Graywind (age: 38)
Innate traits: competent, forbearing, self-discipline, impressionable 
Linene runs a trading post.


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


You are creating a new agent: Harbin Wester
Name: Harbin Wester (age: 40)
Innate traits: pompous
Harbin is the townmaster of Phandalin. A pompous, old food. Phandalin has no functioning government, but the townsfolk elect someone to serve as townmaster each year. The townmaster serves as a judge in minor disputes and keeps any records that need to be kept.


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


You are creating a new agent: Terrill Bloodscar
Name: Terrill Bloodscar (age: 16)
Innate traits: ignorant
Terrill is a human ruffian. He wears a grimy scarlet cloak. He is a member of the Redbrand ruffians. He doesn't like adventurers, and wants to rob and kill them.


In [6]:
global_time = 0
for repeats in range(5):
  global_time += 1
  print("In global time", global_time)
  action_prompts = {}
  action_results = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    # for name in people:
      # prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
    people_description = []
    for i in people: 
      if i not in action_results: # initialize action results as the observations
        action_results[i] = agents[i].get_summary(force_refresh=False)
      people_description.append(i+': '+ action_results[i])


    for i in people: # add observation to memory and react
      observation = "You are {}.You are currently in {} with the following description: {}. \
      It is currently {}:00. The following people are in this area: {}. You can interact with them.". \
      format(i, location, town_areas[location], str(global_time), ', '.join(people))

      observation += ' You know the following about people: ' + '. '.join(people_description)

      print("for people", i)
      print("the observation is:", observation)

      agents[i].memory.add_memory(observation)
      _, reaction = agents[i].generate_reaction(observation)
      action_results[i] = reaction
      print(reaction)

In global time 1
for people Toblen Stonehill
the observation is: You are Toblen Stonehill.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Toblen Stonehill, Daran Edermath, Linene Graywind, Harbin Wester, Terrill Bloodscar. You can interact with them. You know the following about people: Toblen Stonehill: Name: Toblen Stonehill (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Toblen owns a trading post.. Daran Edermath: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.. L

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/langchain/schema/vectorstore.py:313: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content="You are Toblen Stonehill.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Toblen Stonehill, Daran Edermath, Linene Graywind, Harbin Wester, Terrill Bloodscar. You can interact with them. You know the following about people: Toblen Stonehill: Name: Toblen Stonehill (age: 69)\nInnate traits: mean, gossipy, arrogant, selfish\nToblen owns a trading post.. Daran Edermath: Name: Daran Edermath (age: 55)\nInnate traits: Stubborn, upright, inarticulate, calm\nDaran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of

relevant_mem is: - Toblen Stonehill is a grumpy, arrogant, and selfish old man who owns a trading post in Phandalin. He is known for his mean and gossipy nature, and has a tendency to speak his mind without thinking about the consequences.
- Daran Edermath is a retired adventurer who lives in a tidy little cottage beside an apple orchard. He is a fit, silver-haired half-elf well over a hundred years old, and served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. He is stubborn, upright, and inarticulate, and has a calm demeanor.
- Linene Graywind is a competent and forbearing woman who runs a trading post in Phandalin. She is impressionable and has a self-discipline, and is well-respected by the townsfolk.
- Harbin Wester is the pompous townmaster of Phandalin, who serves as a judge in minor disputes and keeps any records that need to be kept. He is an old food, and has a tendency to speak in a pompous manner.
- Terrill Bloodscar is a huma

Token indices sequence length is longer than the specified maximum sequence length for this model (1300 > 1024). Running this sequence through the model will result in indexing errors


Toblen Stonehill Toblen grumbles to himself, "Another day in this godforsaken town. Can't these people just leave me alone?"
for people Daran Edermath
the observation is: You are Daran Edermath.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 1:00. The following people are in this area: Toblen Stonehill, Daran Edermath, Linene Graywind, Harbin Wester, Terrill Bloodscar. You can interact with them. You know the following about people: Toblen Stonehill: Name: Toblen Stonehill (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Toblen owns a trading post.. Daran Edermath: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon 

...

relevant_mem is: - Terrill Bloodscar is a member of the Redbrand ruffians, a group of thieves and criminals that operate in the area around Phandalin. 
- Terrill has a personal problem with adventurers, and wants to rob and kill them.
- Terrill is currently in Phandalin Town Square, along with several other NPCs, including the townmaster Harbin Wester.

Entity=Terrill Bloodscar.


/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Terrill Bloodscar said "what to say"
In global time 3
for people Toblen Stonehill
the observation is: You are Toblen Stonehill.You are currently in Phandalin Town Square with the following description: Town square of the town of Phandalin..       It is currently 3:00. The following people are in this area: Toblen Stonehill, Daran Edermath, Linene Graywind, Harbin Wester, Terrill Bloodscar. You can interact with them. You know the following about people: Toblen Stonehill: Name: Toblen Stonehill (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Toblen owns a trading post.. Daran Edermath: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neve

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/gen

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.12 GiB. GPU 0 has a total capacty of 23.68 GiB of which 2.74 GiB is free. Including non-PyTorch memory, this process has 20.92 GiB memory in use. Of the allocated memory 19.37 GiB is allocated by PyTorch, and 1.31 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF